<font size="4">
    
# MLIB - Deep Learning library

## Structure

* modular OO network with a central class as interface

* no maxpool, but instead filter with stride

* mostly Matrix Ops for everything but the convolution layer, there im2col

* images stored in columns

## Problem

* dataset from an Intel ML challenge, now on kaggle

* contains 17000 labeled images from streets, mountains, seas, buildings, forests, glaciers

* bigger MNIST

## Approach

* started with far to big network, which led to crashes

* began from new and took a really small network and sample size

* scaled up from there

  ## Hyperparameter

* Batch Size: 128

* Layers: 6

    * Convolution Layer: 4
    
    * FullyConnected Layer: 2
    
* MultiClassCrossEntropy Loss

* Adam with standard values

* 873840 learnable parameter

* Similar to LeNet, VGGNet: Filter -> Pooling -> Filter -> Pooling

## Observations

* start small to test, then expand

* pixel dimension > 50 is not worth it (for my system/implementation)

</font>

In [ ]:
import MlibWrapper as mlib
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
from PIL import Image
from random import randint
from math import floor
import pickle

In [ ]:
# Change input pixel dimensions here
img_dim=50

# Max number of images
img_train_num=14034
img_test_num=3000

In [ ]:
# import images
def get_images(directory,number):
    images=np.empty([img_dim*img_dim*3,number],dtype=np.single,order='F')    
    labels=np.empty([number],dtype=np.intc,order='F')
    label=0
    position=0

    path=Path(directory)
    
    for folder in path.iterdir():
        print(folder)
        if folder.name == 'buildings':
            label = 0
        elif folder.name == 'forest':
            label = 1
        elif folder.name == 'glacier':
            label = 2
        elif folder.name == 'mountain':
            label = 3
        elif folder.name == 'sea':
            label = 4
        elif folder.name == 'street':
            label = 5
        else:
            print("Error")

        img_count=0
        for image_file in folder.iterdir():
            # distribute images evenly -> block if one class reaches 1/6 of all images
            if (img_count == number/6):
                break
            img=Image.open(str(image_file))
            img=img.resize((img_dim,img_dim),Image.BILINEAR)
            # normalize image data
            img=np.asarray((img))/255.0
            images[:,position]=np.reshape(img,(img_dim*img_dim*3),order='F')
            
            labels[position]=label
            position=position+1
            img_count=img_count+1

    print("Finished importing")
    # permutation of data
    idx = np.random.permutation(len(labels))
    return (images[:,idx], labels[idx])

# get class name for label
def get_class_label(class_code):
    labels = {0:'buildings', 1:'forest', 2:'glacier', 3:'mountain', 4:'sea', 5:'street'}
    return labels[class_code]


In [ ]:
# The number of images to import and train/test on (has to be divisible by 6)
img_train_num=9900
img_test_num=1200
# Set the correct data location for the inputs
train_images, train_labels = get_images('../data/seg_train/',img_train_num)
test_images, test_labels = get_images('../data/seg_test',img_test_num)

In [ ]:
print("Shape of testing Images:",train_images.shape)
print("Shape of testing Labels:",train_labels.shape)
print("Shape of training Images:",test_images.shape)
print("Shape of training Labels:",test_labels.shape)

In [ ]:
# example picture
pic=9
print(get_class_label(train_labels[pic]))
plt.imshow(np.reshape(train_images[:,pic],(img_dim,img_dim,3),order='F'))
print(train_images[:,pic])

In [ ]:
f,ax = plt.subplots(2,2)
f.subplots_adjust(0,0,3,3)
for i in range(0,2,1):
    for j in range(0,2,1):
        rnd_number = randint(0,len(train_images[0,:]))
        ax[i,j].imshow(np.reshape(train_images[:,rnd_number],(img_dim,img_dim,3),order='F'))
        ax[i,j].set_title(get_class_label(train_labels[rnd_number]))
        ax[i,j].axis('off')

In [ ]:
# The used network architecture
# 6 Layers, 4 Conv (Relu+UniformHe), 2 FC (Tanh+Softmax,UniformXavier), MultiClassCrossEntropy Loss, Adam, 873840 parameter
# Similar to LeNet, VGGNet: Filter -> Pooling -> Filter -> Pooling
NN = mlib.NeuralNetwork()
NN.use_multiclass_loss()
NN.add_conv_layer(50,50,3,3,3,12,1,0)
NN.add_conv_layer(48,48,12,2,2,18,2,0)
NN.add_conv_layer(24,24,18,3,3,36,1,0)
NN.add_conv_layer(22,22,36,2,2,54,2,0)
NN.add_fc_layer(11*11*54,128)
NN.add_output_layer(128,6)
NN.layer_size()
NN.check_network(img_dim*img_dim*3)

In [ ]:
# set visualization parameters 0
accuracy=[]
error=[]
val_accuracy=[]
val_error=[]

In [ ]:
# train the network, only 1/3 of the validation set is used each time for validation
epochs=10
batch_size=128
iterations=floor(img_train_num/batch_size)
print("Iterations per epoch: {}".format(iterations))
size = int(img_test_num/3)

for j in range(0,epochs):
    print("Epoch: {}".format(j))
    for i in range(0,iterations):
        #print("{}".format(i),end = '')
        NN.train_batch(train_images[:,i*batch_size:(i+1)*batch_size],train_labels[i*batch_size:(i+1)*batch_size])
        if(j % 1 ==0 and i ==0):
            #print("Iteration: {}".format(i))
            acc=NN.get_current_accuracy(train_labels[i*batch_size:(i+1)*batch_size])
            print("Accuracy: {}".format(acc))
            err=NN.get_current_error(train_labels[i*batch_size:(i+1)*batch_size])
            print("Error: {}".format(err))
            accuracy.append(acc)
            error.append(err)
            
            k = epochs % 3            
            NN.feed_forward_py(test_images[:,k*size:(k+1)*size])
            acc=NN.get_current_accuracy(test_labels[k*size:(k+1)*size])
            print("Val Accuracy: {}".format(acc))
            err=NN.get_current_error(test_labels[k*size:(k+1)*size])
            print("Val Error: {}".format(err))
            val_accuracy.append(acc)
            val_error.append(err)
                

In [ ]:
# load network weights and graph/visualization data from previously pickelt data (see cell below)
with open('../weights.pickle', 'rb') as handle:
    weights = pickle.load(handle)
with open('../bias.pickle', 'rb') as handle:
    bias = pickle.load(handle)
with open('../graphs.pickle', 'rb') as handle:
    accuracy,error,val_accuracy,val_error =pickle.load(handle)
with open('../info.pickle','rb') as handle:
    info = pickle.load(handle)
print(info)
    
for i in range(0,NN.layer_size()):
    NN.set_layer_weights(weights[i],i)
    NN.set_layer_bias(bias[i],i)

In [ ]:
# summarize history for accuracy
plt.plot(accuracy)
plt.plot(val_accuracy)
plt.title('network accuracy')
plt.ylabel('accuracy')
plt.xlabel('epochs')
plt.legend(['train', 'test'], loc='upper left')
plt.show()
# summarize history for loss
plt.plot(error)
plt.plot(val_error)
plt.title('network loss')
plt.ylabel('loss')
plt.xlabel('epochs')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

In [ ]:
NN.feed_forward_py(test_images[:,0:300])
current_pred=NN.get_current_prediction()
prediction=NN.get_predicted_classes(current_pred)
f,ax = plt.subplots(2,2)
f.subplots_adjust(0,0,3,3)
for i in range(0,2,1):
    for j in range(0,2,1):
        rnd_index = randint(0,len(prediction))
        ax[i,j].imshow(np.reshape(test_images[:,rnd_index],(img_dim,img_dim,3),order='F'))
        ax[i,j].set_title("pred: {}, real: {}".format(get_class_label(prediction[rnd_index]),get_class_label(test_labels[rnd_index])))
        ax[i,j].axis('off')

In [ ]:
# store network weights and graph/visualization data
weights=[]
bias=[]
for i in range(0,NN.layer_size()):
    weights.append(NN.get_layer_weights(i))
    bias.append(NN.get_layer_bias(i))
    
with open('../weights.pickle', 'wb') as handle:
    pickle.dump(weights, handle, protocol=pickle.HIGHEST_PROTOCOL)
with open('../bias.pickle', 'wb') as handle:
    pickle.dump(bias, handle, protocol=pickle.HIGHEST_PROTOCOL)
with open('../graphs.pickle', 'wb') as handle:
    pickle.dump([accuracy,error,val_accuracy,val_error], handle, protocol=pickle.HIGHEST_PROTOCOL)
with open('../info.pickle', 'wb') as handle:
    pickle.dump({'img_dim':img_dim,'img_train_num':img_train_num,'img_test_num':img_test_num,'batch_size':batch_size,'epochs':len(accuracy)}, handle, protocol=pickle.HIGHEST_PROTOCOL)

